In [ ]:
import numpy as np
import pandas as pd
import gc
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Threshold = 35 # % unit (from 0 to 100)

In [ ]:
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Blend

In [ ]:
df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')

In [ ]:
names,preds = [],[]

In [ ]:
!python ../usr/lib/fastai_reduce4_elu_resnet101/fastai_reduce4_elu_resnet101.py

In [ ]:
!python ../usr/lib/fastai_elu_reduce2_resnet101/fastai_elu_reduce2_resnet101.py

In [ ]:
for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    idx = row['id']
    #blend
    pred1 = np.load(f"./pred_{idx}_reduce2.npz")['arr_0'].astype(np.uint8)
    pred2 = np.load(f"./pred_{idx}_reduce4.npz")['arr_0'].astype(np.uint8)
    mask =  (pred1 + pred2) > 2 * Threshold
    #convert to rle
    rle = rle_encode_less_memory(mask)
    names.append(idx)
    preds.append(rle)
    del mask, pred1, pred2, rle
    gc.collect()

In [ ]:
df = pd.DataFrame({'id':names,'predicted':preds})
df.to_csv('submission.csv',index=False)

In [ ]:
df